In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import yaml
import omegaconf
from datetime import datetime

from typing import Union, List
from copy import deepcopy
from tdc.utils import retrieve_benchmark_names

from graphium.config._loader import (
    load_datamodule,
    load_metrics,
    load_architecture,
    load_predictor,
    load_trainer,
    save_params_to_wandb,
    load_accelerator,
    load_yaml_config,
)

# Fine-tuning on the TDC ADMET benchmarking group

[TDC](https://tdcommons.ai/) hosts a variety of ML-ready datasets and benchmarks for ML for drug discovery. The [TDC ADMET benchmarking group](https://tdcommons.ai/benchmark/admet_group/overview/) is a popular collection of benchmarks for evaluating new _foundation models_ (see e.g. [MolE](https://arxiv.org/abs/2211.02657)) due to the variety and relevance of the included tasks.

The ADMET benchmarking group is integrated in `graphium` through the `ADMETBenchmarkDataModule` data-module. This notebook shows how to easily fine-tune and test a model using that data-module. 

<div style="background-color: #fff3cd; border-radius: 10px; border-color: #ffeeba; padding: 20px; margin: 20px 0;  color: #856404">
    <b>NOTE:</b> This notebook is still <i>work in progress</i>. While the <b>fine-tuning logic is unfinished</b>, the notebook does demo how one could use the data-module to easily loop over each of the datasets in the benchmarking group and get the prescribed train-test split. Once the fine-tuning logic is finalized, we will finish this notebook and officially provide it as a tutorial within Graphium. 
</div>

In [3]:
# First, let's read the yaml configuration file
with open("../expts/configs/config_tdc_admet_demo.yaml", "r") as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

## Get all TDC benchmark names

In [4]:
benchmarks = retrieve_benchmark_names("admet_group")
len(benchmarks)

22

While there is a total of 22, let's just use two for practicality sake: One regression and one classification task! 

In [5]:
benchmarks = ["caco2_wang", "hia_hou"]

## Initialize all training components per task
**NOTE**: Since we do not have fine-tuning logic, this for now just creates a new model. Ultimately, we will want to use fine-tuning code to evaluate how well the pre-trained model transfers to downstream tasks. 

In [6]:
def training_testing_loop(cfg):
    """
    Simple loop to train a model from scratch and test it. 
    """
    
    # Initialize object from config
    cfg, accelerator_type = load_accelerator(cfg)
    datamodule = load_datamodule(cfg, accelerator_type)
    model_class, model_kwargs = load_architecture(cfg, in_dims=datamodule.in_dims)
    metrics = load_metrics(cfg)
    
    # Prepare data
    datamodule.prepare_data()
    
    # Initialize the predictor
    predictor = load_predictor(
        cfg,
        model_class,
        model_kwargs,
        metrics,
        datamodule.get_task_levels(),
        accelerator_type,
        datamodule.featurization,
        datamodule.task_norms
    )
    
    # Initialize the trainer
    date_time_suffix = datetime.now().strftime("%d.%m.%Y_%H.%M.%S")
    trainer = load_trainer(cfg, "tdc-admet", accelerator_type, date_time_suffix)
        
    # Train
    predictor.set_max_nodes_edges_per_graph(datamodule, stages=["train", "val"])
    trainer.fit(model=predictor, datamodule=datamodule)
    
    # Test
    predictor.set_max_nodes_edges_per_graph(datamodule, stages=["test"])
    results = trainer.test(model=predictor, datamodule=datamodule)
    
    return results


In [7]:
def filter_cfg_based_on_benchmark_name(config, names: Union[List[str], str]):
    """
    Filter a base config for the full TDC ADMET benchmarking group to only 
    have settings related to a subset of the endpoints
    """
    
    if config["datamodule"]["module_type"] != "ADMETBenchmarkDataModule":
        raise ValueError("You can only use this method for the `ADMETBenchmarkDataModule`")
        
    if isinstance(names, str):
        names = [names]
    
    def _filter(d):
        return {k: v for k, v in d.items() if k in names}
         
    cfg = deepcopy(config)
    
    # Update the datamodule arguments
    cfg["datamodule"]["args"]["tdc_benchmark_names"] = names
    
    # Filter the relevant config sections
    cfg["architecture"]["task_heads"] = _filter(cfg["architecture"]["task_heads"])
    cfg["predictor"]["metrics_on_progress_bar"] = _filter(cfg["predictor"]["metrics_on_progress_bar"])
    cfg["predictor"]["loss_fun"] = _filter(cfg["predictor"]["loss_fun"])
    cfg["metrics"] = _filter(cfg["metrics"])
    
    return cfg

In [ ]:
results = {}

for name in benchmarks: 
    
    # Run the training-testing loop
    cfg = filter_cfg_based_on_benchmark_name(config, name)
    benchmark_results = training_testing_loop(cfg)
    
    # Extract the main metric from the config
    metric = cfg["predictor"]["metrics_on_progress_bar"][name][0]
    key = f"graph_{name}/{metric}/test"
    results[f"{name}/{metric}"] = benchmark_results[0][key]

In [9]:
print(omegaconf.OmegaConf.to_yaml(results))

caco2_wang/mae: 2.312683582305908
hia_hou/auroc: 0.7008230686187744



The End. 